# 6.2 Binary Data Formats (二进制数据格式)

最简单的以二进制的格式来存储数据的方法（也被叫做serialization，序列化），就是用python内建的pickle。所有的pandas object都有一个to_pickle方法，可以用来存储数据为pickle格式：

In [52]:
import pandas as pd
import numpy as np

In [53]:
frame = pd.read_csv('../../examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [54]:
frame.to_pickle('../../examples/frame_pickle')

In [55]:
!ls ../../examples/

array_archive.npz          macrodata.csv
array_compressed.npz       mydata.csv
array_ex.txt               mydata.h5
cprof_example.py           mydata.sqlite
csv_mindex.csv             out.csv
ex1.csv                    pro_mod.py
ex1.xlsx                   segismundo.txt
ex2.csv                    sink.txt
ex2.xlsx                   some_array.npy
ex3.csv                    spx.csv
ex3.txt                    stinkbug.png
ex4.csv                    stock_px.csv
ex5.csv                    stock_px_2.csv
ex6.csv                    test_file.csv
ex7.csv                    tips.csv
example.json               tmp.txt
fdic_failed_bank_list.html tseries.csv
frame_pickle               volume.csv
ipython_bug.py             yahoo_price.pkl
ipython_bug2.py            yahoo_volume.pkl


用内建的pickle可以直接读取任何pickle文件，或者直接用pandas.read_pickle:

In [56]:
pd.read_pickle('../../examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


注意：pickle只推荐用于短期存储。因为这种格式无法保证长期稳定；比如今天pickled的一个文件，可能在库文件更新后无法读取。

python还支持另外两种二进制数据格式：HDF5和MessagePack。下面会介绍一个HDF5，但是我们鼓励你多尝试一个不同的文件格式，看看他们能有多快，是否符合你数据分析的要求。另外一些可用的存储格式有：bcolz 和 Feather。

# 1 Using HDF5 Format

HDF5格式是用来存储大量的科学数组数据的。这种格式还能用于其他一些语言。其中HDF表示hierarchical data format。每一个HDF5格式能存储多个数据集，并支持metadata。

> 元数据(meta data)——“data about data” 关于数据的数据，一般是结构化数据（如存储在数据库里的数据，规定了字段的长度、类型等）。元数据是指从信息资源中抽取出来的用于说明其特征、内容的结构化的数据(如题名,版本、出版数据、相关说明,包括检索点等)，用于组织、描述、检索、保存、管理信息和知识资源。

HDF5 支持多种压缩模式的on-the-fly compression（即时压缩），能让数据中一些重复的部分存储地更有效。HDF5对于处理大数据集是一个很好的选择，因为他不会把所有数据一次性读取到内存里，我们可以从很大的数组中有效率地读取一小部分。

能用PyTables或h5py来访问HDF5数据，pandas也有提供一个high-level的交互界面。HDFStore类像dict一样能用来处理low-level细节：


In [57]:
frame = pd.DataFrame({'a': np.random.randn(100)})

store = pd.HDFStore('../../examples/mydata.h5')

# 需要 conda istall pytables （依赖）
# 没有的报错信息 ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [58]:
frame.head()

,a
0,-0.178550
1,0.718144
2,-1.544954
3,-0.273620
4,0.460133


In [59]:
cat ../../examples/mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


In [60]:
store['obj1'] = frame

store['obj1_col'] = frame['a']

store

<class 'pandas.io.pytables.HDFStore'>
File path: ../../examples/mydata.h5

HDF5中的object能用像dict一样的API来提取：

In [61]:
store['obj2']

,a
0,-1.772343
1,0.645514
2,-0.535837
3,0.547282
4,-0.157336
...,...
95,-0.845503
96,-0.707685
97,-1.079555
98,1.690901


HDFStore支持两种存储架构，fixed和table。后者通常更慢一些，但支持查询操作：

In [62]:
store.put('obj2', frame, format='table')

In [63]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-1.363255
11,1.238999
12,-0.650884
13,-1.137618
14,1.111589
15,-0.130655


put是存储的另一种写法，类似于之前的`store['obj2'] = frame`，但这种写法能让我们设置存储格式。

pandas.read_hdf函数也很方便：

In [65]:
# frame.to_hdf('../../examples/mydata.h5', 'obj3', format='table')
frame.to_hdf('../../examples/mydata.h5', 'obj2',  format='table')

In [71]:
pd.read_hdf('../../examples/mydata.h5', 'obj2', mode = 'r+')
# r+是在存在时追加的参数，因为已经open，如果默认r会报错已经open，后续再研究细节

,a
0,-0.178550
1,0.718144
2,-1.544954
3,-0.273620
4,0.460133
...,...
95,0.961814
96,0.425686
97,0.523406
98,1.568061


In [74]:
pd.read_hdf('../../examples/mydata.h5', 'obj2', mode = 'r+', where=['index < 5'])

,a
0,-0.178550
1,0.718144
2,-1.544954
3,-0.273620
4,0.460133


注意：如果我们是把数据存在远端服务器上，比如Amazon S3或HDFS，使用一些为分布式存储实际的二进制格式会更适合一些，比如Apache Parquet。

如果是在本地处理很大数据量的话，推荐尝试PyTables和h5py看是否符合你的要求。因为很多数据分析问题都受限于I/O，所以用HDF5这样的工具能加快应用。

注意：HDF5不是数据库（database）。它最适合一次写入，多次读取的数据库。尽管数据可以在任何时间多次写入一个文件，如果多个使用者同时写入的话，文件会被破坏。

# 2 Reading Microsoft Excel Files（读取微软的excel文件）

pandas支持读取表格型数据（excel 2003或更高）文件，使用ExcelFile class或pandas.read_excel函数。这些工具需要一些富家的包xlrd和openpyxl来分别读取XLS和XLSX文件。你可以通过pip或conda来安装。

使用ExcelFile，创建一个instance，通过给xls或xlsx一个路径：

In [76]:
xlsx = pd.ExcelFile('../../examples/ex1.xlsx')

保存在sheet里的数据，可以通过parse来读取为DataFrame：

In [77]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要读取一个文件中的多个sheet，用ExcelFile会更快。但让，你也能把文件名直接传递给pandas.read_excel:

In [79]:
frame = pd.read_excel('../../examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要把pandas数据写为Excel格式，你必须先创建一个ExcelWrite，然后用to_excel方法：

In [81]:
writer = pd.ExcelWriter('../../examples/ex2.xlsx')
# 需要安装 openpyxl

In [82]:
frame.to_excel(writer, 'Sheet1')

In [83]:
writer.save()

如果不适用ExcelWriter的话，可以直接传给to_excel一个path：

In [84]:
frame.to_excel('../../examples/ex2.xlsx')